In [12]:
import pandas as pd
import math
from decimal import *
import numpy as np

In [13]:
dfAden = pd.read_csv('../data/aden.csv',sep=';')

# Ajustement

In [14]:
dfAden['Annee (0)'] = dfAden['Annee (0)'].astype(int)
dfAden['jour (0)'] = dfAden['jour (0)'].astype(int)
dfAden['mois (0)'] = dfAden['mois (0)'].astype(int)

dfAden['descriptif']=dfAden.apply(lambda row: row['descriptif'].replace('\r', ''), axis=1)

# Affectation grandes zones

Dans cette section, on définit les grandes zones :
- Golfe d'aden
- Est
- Sud

In [15]:
golfe_min_lat=10
golfe_max_lat=17
golfe_min_lon=40
golfe_max_lon=56

arabie_min_lon=56
golfe_min_lat=10

def choose_large_area(latitude,longitude):
    if latitude<golfe_max_lat and latitude>golfe_min_lat and longitude<=golfe_max_lon and longitude>golfe_min_lon :
        return"G"
    elif latitude>golfe_min_lat and longitude>arabie_min_lon:
        return"A"
    else:
        return"I"
    
#print choose_large_area(11,41)

dfAden['large_area'] = dfAden.apply(lambda row: choose_large_area(row['latitude'],row['longitude']), axis=1)

dfAden.to_csv("../data/aden.csv",sep=';', index=False)

# Affectation des sous zones

In [16]:
dfAden = pd.read_csv('../data/aden.csv',sep=';')

Cette fonction renvoie deux tableaux contenant le découpage en zones

In [17]:
def get_grid_area(min_lat,max_lat,min_lon,max_lon,accuracy_km=60): 
    #accuracy_km est la précision que l'on veut donner
    #un degre de latitude "vaut"  111,11 km
    accuracy_lat=accuracy_km/111.11

    diff_lat=max_lat-min_lat
    diff_lon=max_lon-min_lon

    path_lat=float(diff_lat)/float(accuracy_lat)

    latitude_grid=np.linspace(min_lat, max_lat, path_lat)
    longitude_grid=[]
    longitude_grid.append(golfe_min_lon)

    tab=[]
    for i in range(0,len(latitude_grid)-1):
        mean_lat=float(latitude_grid[i]+latitude_grid[i+1])/2
        km_lon=111.11*math.cos(math.radians(mean_lat))
        accuracy_lon=accuracy_km/km_lon
        tab.append(accuracy_lon) 
        currentLon=longitude_grid[i]+accuracy_lon
        longitude_grid.append(currentLon)

    #on complete jusqu'a golfe_max_lon
    maxCurrent=max(longitude_grid)
    meanPath=np.mean(tab)
    while maxCurrent<max_lon:
        maxCurrent=max(longitude_grid)
        longitude_grid.append(maxCurrent+meanPath)
    
    return latitude_grid.tolist(),longitude_grid


golfe_latitude_grid,golfe_longitude_grid=get_grid_area(golfe_min_lat,golfe_max_lat,golfe_min_lon,golfe_max_lon)


print len(golfe_latitude_grid)
print len(golfe_longitude_grid)
print golfe_latitude_grid
print golfe_longitude_grid

12
31
[10.0, 10.636363636363637, 11.272727272727273, 11.90909090909091, 12.545454545454545, 13.181818181818182, 13.818181818181818, 14.454545454545453, 15.09090909090909, 15.727272727272727, 16.363636363636363, 17.0]
[40, 40.548881779729705, 41.098909650518635, 41.65015655057219, 42.20269620596189, 42.756603182819944, 43.31195294089674, 43.86882188858008, 44.42728743947935, 44.987428070682796, 45.549323382801425, 46.11305416191886, 46.668786358456941, 47.224518554995022, 47.780250751533103, 48.335982948071184, 48.891715144609265, 49.447447341147345, 50.003179537685426, 50.558911734223507, 51.114643930761588, 51.670376127299669, 52.226108323837749, 52.78184052037583, 53.337572716913911, 53.893304913451992, 54.449037109990073, 55.004769306528154, 55.560501503066234, 56.116233699604315, 56.671965896142396]


On utilise ce découpage pour créer deux colonnes : lat_area et lon_area

In [18]:
def affectation_zone(latitude,longitude,latitude_grid,longitude_grid,lat_bool):
    #print 'latitude : '+ str(latitude)
    #print 'longitude : '+ str(longitude)
    #print latitude_grid
    idx_zone_lat=None
    idx_zone_lon=None
    
    for i in range(0,len(latitude_grid)-1):        
        if latitude<=latitude_grid[i+1] and latitude>latitude_grid[i]:
            idx_zone_lat=i

    for i in range(0,len(longitude_grid)-1):
        if longitude<=longitude_grid[i+1] and longitude>longitude_grid[i]:
            idx_zone_lon=i

    #print idx_zone_lon
    #print idx_zone_lat
    if lat_bool:
        return idx_zone_lat
    else:
        return idx_zone_lon

Test

In [19]:
latitude=-5.23333333333
longitude=42.6166666667
idx_zone_lat=affectation_zone(latitude,longitude,golfe_latitude_grid,golfe_longitude_grid,True)
idx_zone_lon=affectation_zone(latitude,longitude,golfe_latitude_grid,golfe_longitude_grid,False)
print idx_zone_lat
print idx_zone_lon

None
4


In [20]:
dfAden['lat_area'] = dfAden[dfAden['large_area']=="G"].apply(lambda row: affectation_zone(row['latitude'],row['longitude'],golfe_latitude_grid,golfe_longitude_grid,True), axis=1)
dfAden['lon_area'] = dfAden[dfAden['large_area']=="G"].apply(lambda row: affectation_zone(row['latitude'],row['longitude'],golfe_latitude_grid,golfe_longitude_grid,False), axis=1)

In [21]:
dfAden[dfAden['large_area']=="G"].head(3)

,Annee (0),jour (0),mois (0),dist cote (nq),zone,lat (0),long(0),type(0),pavillon,statut de l'attaque(0),...,quinzaine_week,newDescriptif,Guns,mousson,distance au port,distance_shorline,lat_area,lon_area,large_area,distance a la base de pirate
23,2010,28,10,NaN,BCD,12:08N,054:25E,Dhow,Not Stated,Steaming,...,20,pirates hijacked a dhow.,0,0,534.679104,35.872306,3,25,G,346.662210
25,2010,30,10,NaN,BCD,15:06N,055:58E,Bulk Carrier,Malta,Steaming,...,20,armed pirates attacked and boarded the ship un...,0,0,292.330113,233.517911,8,28,G,626.662668
42,2010,1,1,NaN,BCD,12:31N,047:18E,Chemical Tanker,Singapore,Steaming,...,1,pirates hijacked a tanker underway and took he...,0,0,259.217741,118.411436,3,13,G,244.881989


### On ecrit un fichier par zone

In [22]:
dfAden[dfAden['large_area']=="G"].to_csv("../data/aden_golfe.csv",sep=';', index=False)